# Preprocess Raw Data to be ready to Train

## Purpose

This notebook has the purpose to prepare the dataset to be ready use for the Deep Neural Networks Architecture that will be use for classify and compare with the paper PerceptSent.

## Import Libraries

In [2]:
import os
import shutil
import pandas as pd
import nltk
# import gensim
from tqdm.notebook import tqdm
# Deep learning packages
import torch
from sklearn.model_selection import train_test_split

from collections import Counter
# from textaugment import Word2vec
# from textaugment import Fasttext
# from textaugment import Wordnet
# from textaugment import Translate
# from textaugment import Word2vec, Fasttext
# from textaugment import Translate

# Visualization packages
import matplotlib.pyplot as plt
import seaborn

In [3]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/neemias/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/neemias/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/neemias/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Load DataFrame

In [ ]:
!ls ..

In [5]:
# df = pd.read_csv("../data/raw/image_caption_mini_GPT4.csv") # captioning data for miniGPT-4
# df = pd.read_csv("../data/raw/image_caption_blip.csv") # captioning data for Blip Large model
# df_mini = pd.read_csv("../data/raw/image_caption_minigpt4_completed.csv") # captioning data for miniGPT-4
# df = pd.read_csv("../data/raw/image_caption_gpt4_openai.csv", delimiter=';') # captioning data for miniGPT-4
df = pd.read_csv("../data/raw/image_caption_deepseek.csv") # captioning data for miniGPT-4
# df["url"] = ["https://drive.google.com/uc?export=view&id="+str(id) for id in df["id"]]
display(df.head())
display(df.tail())

,id,text
0,17rbdLJ5aNayZk9lK8wnVeICSgwJxLVA_,The image depicts a scene of a street with a w...
1,13rljxGdgMVqhS9Shw-QJdvAsrUtw6uG3,The image depicts a rustic scene with a large ...
2,150R6i7Id13s-D4dbuxcDnKDcA7sqjo7e,The image depicts a beautiful winter landscape...
3,11DOuFmByYWPoQc2QyFxzxcb1da83nIqF,The image depicts an abandoned and dilapidated...
4,135mA2oT4gSOf3qcrmsFj5OE7kL406F4T,The image depicts a nighttime scene with a per...


,id,text
4995,1w0roE1aw1yjQ8oY0__9t93nO1ARMYfV0,The image depicts a scene with a concrete stru...
4996,1iXaZO6ytrrK2TyCCBwrjxiZ5ENHFr1b3,The image depicts a scene from what appears to...
4997,1ijXLFVUpmqbBhlACprg0b9NpEoznSMxM,"The image depicts a dark, rocky surface with a..."
4998,1acey5joN49lEijBO-6VCVYDqbvJt67AI,The image depicts a vibrant graffiti mural pai...
4999,1-f8C5VH-0TsCfr0DkrzUVp8Pub9xykXp,The image depicts a section of a building's


In [6]:
df.shape

(5000, 2)

In [6]:
# df["id"] = df["image_path"].apply(lambda x: x.split('/')[-1].split('.')[0]) # for miniGPT4

In [ ]:
df.shape

In [ ]:
df.head()

In [9]:
# df["caption"] = df["caption"].apply(lambda x: x.replace("The image shows", '').replace("This is", '').replace("This image", '').replace("The image", '').replace("\u200b", '').replace("\n", ' ').replace("<Img>", ''))

In [10]:
# model = gensim.models.KeyedVectors.load_word2vec_format('../models/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [7]:
import json

f = open("../data/raw/dataset.json")
 
# returns JSON object as 
# a dictionary
data = json.load(f)

f.close()

In [26]:
sentiment_dict = {
    "SlightlyNegative": "Negative",
    # "SlightlyNegative": "SlightlyNegative",
    "SlightlyPositive": "Positive",
    # "SlightlyPositive": "SlightlyPositive",
    "Neutral": "Neutral",
    "Positive": "Positive",
    "Negative": "Negative"
}

In [ ]:
def ia_calculation(sg):
    """
    Calcula a métrica IA (Image Agreement) com base no vetor de sentimento agrupado (sg).

    Parâmetros:
    sg (list ou tuple): Vetor de sentimento agrupado, onde cada elemento representa o número de votos para uma categoria de sentimento.

    Retorna:
    float: A métrica IA.
    """
    total_avaliadores = len(sg)
    max_votos = max(sg)
    IA = max_votos / total_avaliadores
    return IA

sentiment_data = {}

for id, caption in tqdm(zip(df["id"], df["caption"]), total=len(df), desc="Create supervised dataset"):
    sentiment_data[id] = {
        "sparse sentiment": [],
        "cluster sentiment": [],
        "perceptions": [],
        "caption": caption,
        "image_agreement": float,
    }

simple_sentiment = {
    "Positive": "Positive",
    "SlightlyPositive": "Positive",
    # "SlightlyPositive": "SlightlyPositive",
    "Neutral": "Neutral",
    "SlightlyNegative": "Negative",
    # "SlightlyNegative": "SlightlyNegative",
    "Negative": "Negative"
}
# sentiment_idx = {
#     "Positive": 4,
#     "SlightlyPositive": 3,
#     "Neutral": 2,
#     "SlightlyNegative": 1,
#     "Negative": 0
# }
sentiment_idx = {
    "Positive": 2,
    "Neutral": 0,
    "Negative": 1
}


for samples in tqdm(data["tasks"], desc="Image perceptions"):
    for sample in samples["images"]:
        id = sample["id"]
        sentiment = sample["sentiment"]
        # perceptions = ', '.join([str(per) for per in sample["perceptions"]])
        perceptions = [str(per) for per in sample["perceptions"]]
        
        if id in sentiment_data:
            # sentiment = [simple_sentiment[sent] for sent in sentiment]
            sentiment_data[id]["sparse sentiment"].append(sentiment)
            sentiment_data[id]["cluster sentiment"].append(simple_sentiment[sentiment])
            for perception in perceptions:
                sentiment_data[id]["perceptions"].append(perception)

for id in tqdm(sentiment_data, desc="Image agreement calculation"):
    sentiment = sentiment_data[id]["sparse sentiment"]
    sg = [0 for _ in range(len(sentiment))]
    counter = Counter(sentiment)
    for key in counter:
        sg[sentiment_idx[key]] = counter[key]
    sentiment_data[id]["image_agreement"] = ia_calculation(sg)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
data = {
    "text": [],
    "target": [],
}

# sentiment_values = {
#     "Negative": 0,
#     "SlightlyNegative": 1,
#     "Neutral": 2,
#     "SlightlyPositive": 3,
#     "Positive": 4,
# }
sentiment_values = {
    "Negative": 0,
    "Neutral": 0,
    "Positive": 1,
}

for id in tqdm(sentiment_data, desc="Zero-shot-classification progress"):
    caption = sentiment_data[id]["caption"]
    sentiment = list(sentiment_data[id]["cluster sentiment"])
    unique_perceptions = list(set(sentiment_data[id]["perceptions"]))
    # max_value = result[pred]
    ia = sentiment_data[id]["image_agreement"]    

    label = Counter(sentiment)
    counter = Counter(sentiment)
    # print(sentiment)
    # print(label)
    # print(counter)
    most_common_sentiment, frequency = counter.most_common(1)[0]
    if (frequency >= 5): # 3 -> alpha3; 4 -> alpha4; 5 -> alpha5
        data["text"].append(caption)
        data["target"].append(sentiment_values[most_common_sentiment])

In [ ]:
data["text"][0], data["target"][0]

## Analyze the Target Distribution

In [35]:
def plot_sentiment_distribution(df):
    """"
    Plots the distribution of sentiments
    Parameters:
        df(dataframe): DataFrame containing the "sentiment" column.

    Returns:
        None
    """

    mapping = {0: "Negative", 1: "Positive"}
    # mapping = {1: "Negative", 0: "Neutral", 2: "Positive"}
    # mapping = {0: "Negative", 1: "SlightlyNegative", 2: "Neutral", 3: "SlightlyPositive", 4: "Positive"}
    df["sentiment"] = df["sentiment"].map(mapping)
    sentiment_counts = df["sentiment"].value_counts()

    plt.figure(figsize=(6, 4))
    sentiment_counts.plot(kind="bar", color="skyblue")
    plt.title("Sentiment Distribution")
    plt.xlabel("Sentiment")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_sentiment_distribution(pd.DataFrame({"sentiment": data["target"]}))

In [ ]:
data["text"][:5], data["target"][:5]

In [ ]:
len(data["target"])

## Train & Test Split

## Save DataFrames

In [55]:
# train_df.to_csv("../data/train/train.csv", index=False)
# # aug_train_df.to_csv("../data/train/aug_train.csv", index=False)
# val_df.to_csv("../data/validation/val.csv", index=False)
# test_df.to_csv("../data/test/test.csv", index=False)

In [51]:
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha3_p5.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha3_p3.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha3_p2plus.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha3_p2neg.csv", index=False)

In [95]:
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha4_p5.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha4_p3.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha4_p2plus.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha4_p2neg.csv", index=False)

In [39]:
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha5_p5.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha5_p3.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha5_p2plus.csv", index=False)
# pd.DataFrame({"text": data["text"], "sentiment": data["target"]}).to_csv("../data/percept_dataset_alpha5_p2neg.csv", index=False)

# Generate Data to MiniGPT 4 classify sentiment.

In [6]:
import pandas as pd
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

In [8]:
# df = pd.read_csv("../data/raw/image_caption_minigpt4_completed.csv") # captioning data for miniGPT-4
# df = pd.read_csv("../data/raw/image_caption_gpt4_openai.csv", delimiter=';') # captioning data for miniGPT-4
# df["url"] = ["https://drive.google.com/uc?export=view&id="+str(id) for id in df["id"]]
df = pd.read_csv("../data/raw/image_caption_deepseek.csv") # captioning data for miniGPT-4
# df["url"] = ["https://drive.google.com/uc?export=view&id="+str(id) for id in df["id"]]
display(df.head())
display(df.tail())
df.head()

,id,text
0,17rbdLJ5aNayZk9lK8wnVeICSgwJxLVA_,The image depicts a scene of a street with a w...
1,13rljxGdgMVqhS9Shw-QJdvAsrUtw6uG3,The image depicts a rustic scene with a large ...
2,150R6i7Id13s-D4dbuxcDnKDcA7sqjo7e,The image depicts a beautiful winter landscape...
3,11DOuFmByYWPoQc2QyFxzxcb1da83nIqF,The image depicts an abandoned and dilapidated...
4,135mA2oT4gSOf3qcrmsFj5OE7kL406F4T,The image depicts a nighttime scene with a per...


,id,text
4995,1w0roE1aw1yjQ8oY0__9t93nO1ARMYfV0,The image depicts a scene with a concrete stru...
4996,1iXaZO6ytrrK2TyCCBwrjxiZ5ENHFr1b3,The image depicts a scene from what appears to...
4997,1ijXLFVUpmqbBhlACprg0b9NpEoznSMxM,"The image depicts a dark, rocky surface with a..."
4998,1acey5joN49lEijBO-6VCVYDqbvJt67AI,The image depicts a vibrant graffiti mural pai...
4999,1-f8C5VH-0TsCfr0DkrzUVp8Pub9xykXp,The image depicts a section of a building's


,id,text
0,17rbdLJ5aNayZk9lK8wnVeICSgwJxLVA_,The image depicts a scene of a street with a w...
1,13rljxGdgMVqhS9Shw-QJdvAsrUtw6uG3,The image depicts a rustic scene with a large ...
2,150R6i7Id13s-D4dbuxcDnKDcA7sqjo7e,The image depicts a beautiful winter landscape...
3,11DOuFmByYWPoQc2QyFxzxcb1da83nIqF,The image depicts an abandoned and dilapidated...
4,135mA2oT4gSOf3qcrmsFj5OE7kL406F4T,The image depicts a nighttime scene with a per...


In [9]:
import json

f = open("../data/raw/dataset.json")
data_json = json.load(f)

f.close()

In [10]:
def ia_calculation(sg):
    """
    Calcula a métrica IA (Image Agreement) com base no vetor de sentimento agrupado (sg).

    Parâmetros:
    sg (list ou tuple): Vetor de sentimento agrupado, onde cada elemento representa o número de votos para uma categoria de sentimento.

    Retorna:
    float: A métrica IA.
    """
    total_avaliadores = len(sg)
    max_votos = max(sg)
    IA = max_votos / total_avaliadores
    return IA

In [12]:
for approach in range(0, 4):
    if approach == 0: ## P5
        p = 'p5'
        sentiment_dict = {
            "SlightlyNegative": "SlightlyNegative",
            "SlightlyPositive": "SlightlyPositive",
            "Neutral": "Neutral",
            "Positive": "Positive",
            "Negative": "Negative"
        }
        simple_sentiment = {
            "Positive": "Positive",
            "SlightlyPositive": "SlightlyPositive",
            "Neutral": "Neutral",
            "SlightlyNegative": "SlightlyNegative",
            "Negative": "Negative"
        }
        sentiment_idx = {
            "Positive": 4,
            "SlightlyPositive": 3,
            "Neutral": 2,
            "SlightlyNegative": 1,
            "Negative": 0
        }
        sentiment_values = {
            "Negative": 0,
            "SlightlyNegative": 1,
            "Neutral": 2,
            "SlightlyPositive": 3,
            "Positive": 4,
        }
    elif approach == 1: ## P3
        p = 'p3'
        sentiment_dict = {
            "SlightlyNegative": "Negative",
            "SlightlyPositive": "Positive",
            "Neutral": "Neutral",
            "Positive": "Positive",
            "Negative": "Negative"
        }
        simple_sentiment = {
            "Positive": "Positive",
            "SlightlyPositive": "Positive",
            "Neutral": "Neutral",
            "SlightlyNegative": "Negative",
            "Negative": "Negative"
        }
        sentiment_idx = {
            "SlightlyPositive": 2,
            "Positive": 2,
            "Neutral": 0,
            "Negative": 1,
            "SlightlyNegative": 1,
        }
        sentiment_values = {
            "Negative": 1,
            "Neutral": 0,
            "Positive": 2,
        }
    elif approach == 2: ## P2+
        p = 'p2plus'
        sentiment_dict = {
            "SlightlyNegative": "Negative",
            "SlightlyPositive": "Positive",
            "Neutral": "Positive",
            "Positive": "Positive",
            "Negative": "Negative"
        }
        simple_sentiment = {
            "Positive": "Positive",
            "SlightlyPositive": "Positive",
            "Neutral": "Positive",
            "SlightlyNegative": "Negative",
            "Negative": "Negative"
        }
        sentiment_idx = {
            "SlightlyPositive": 0,
            "Positive": 0,
            "Neutral": 0,
            "Negative": 1,
            "SlightlyNegative": 1,
        }
        sentiment_values = {
            "Negative": 1,
            "Neutral": 0,
            "Positive": 0,
        }
    elif approach == 3: ## P2-
        p = 'p2neg'
        sentiment_dict = {
            "SlightlyNegative": "Negative",
            "SlightlyPositive": "Positive",
            "Neutral": "Negative",
            "Positive": "Positive",
            "Negative": "Negative"
        }
        simple_sentiment = {
            "Positive": "Positive",
            "SlightlyPositive": "Positive",
            "Neutral": "Negative",
            "SlightlyNegative": "Negative",
            "Negative": "Negative"
        }
        sentiment_idx = {
            "SlightlyPositive": 1,
            "Positive": 1,
            "Neutral": 0,
            "Negative": 0,
            "SlightlyNegative": 0,
        }
        sentiment_values = {
            "Negative": 0,
            "Neutral": 0,
            "Positive": 1,
        }

    
    sentiment_data = {}

    for id, caption in tqdm(zip(df["id"], df["text"]), total=len(df), desc="Create supervised dataset"):
        sentiment_data[id] = {
            "sparse sentiment": [],
            "cluster sentiment": [],
            "perceptions": [],
            "caption": caption,
            "image_agreement": float,
        }

        
    for samples in tqdm(data_json["tasks"], desc="Image perceptions"):
        for sample in samples["images"]:
            id = sample["id"]
            sentiment = sample["sentiment"]
            # perceptions = ', '.join([str(per) for per in sample["perceptions"]])
            perceptions = [str(per) for per in sample["perceptions"]]
            
            if id in sentiment_data:
                # sentiment = [simple_sentiment[sent] for sent in sentiment]
                sentiment_data[id]["sparse sentiment"].append(sentiment)
                sentiment_data[id]["cluster sentiment"].append(simple_sentiment[sentiment])
                for perception in perceptions:
                    sentiment_data[id]["perceptions"].append(perception)

    for id in tqdm(sentiment_data, desc="Image agreement calculation"):
        sentiment = sentiment_data[id]["sparse sentiment"]
        sg = [0 for _ in range(len(sentiment))]
        counter = Counter(sentiment)
        for key in counter:
            # if p == 'p3':
            #     print(sg)
            #     print(key)
            #     print(sentiment_idx)
            sg[sentiment_idx[key]] = counter[key]
        sentiment_data[id]["image_agreement"] = ia_calculation(sg)
    
    for f in range(3, 6):
    
        data = {
            "text": [],
            "target": [],
            "id": []
        }
        for id in tqdm(sentiment_data, desc="Generate datasets"):
            caption = sentiment_data[id]["caption"]
            sentiment = list(sentiment_data[id]["cluster sentiment"])
            unique_perceptions = list(set(sentiment_data[id]["perceptions"]))
            ia = sentiment_data[id]["image_agreement"]    

            label = Counter(sentiment)
            counter = Counter(sentiment)
            most_common_sentiment, frequency = counter.most_common(1)[0]
            if (frequency >= f): # 3 -> alpha3; 4 -> alpha4; 5 -> alpha5
                data["text"].append(caption)
                data["target"].append(sentiment_values[most_common_sentiment])
                data["id"].append(id)

        pd.DataFrame({"id": data["id"], 
                      "text": data["text"], 
                      "sentiment": data["target"]}).to_csv(
                          f"../data/deepseek/percept_dataset_alpha{f}_{p}.csv", index=False)
    


Create supervised dataset:   0%|          | 0/5000 [00:00<?, ?it/s]

Image perceptions:   0%|          | 0/2500 [00:00<?, ?it/s]

Image agreement calculation:   0%|          | 0/5000 [00:00<?, ?it/s]

Generate datasets:   0%|          | 0/5000 [00:00<?, ?it/s]

Generate datasets:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Image perceptions:   0%|          | 0/2500 [00:00<?, ?it/s]

Image agreement calculation:   0%|          | 0/5000 [00:00<?, ?it/s]

Generate datasets:   0%|          | 0/5000 [00:00<?, ?it/s]

Generate datasets:   0%|          | 0/5000 [00:00<?, ?it/s]

Generate datasets:   0%|          | 0/5000 [00:00<?, ?it/s]